In [1]:
from pyspark.sql import SparkSession
from sedona.utils import SedonaKryoRegistrator
from pyspark.sql.functions import col, udf
from pyspark.sql.functions import expr

import geopandas as gpd
from shapely import Point

In [2]:
# Setup SparkSession and add Sedona
# Sedona enables us to use PostGIS like functions to find intersections

sedona_jar = '/home/jovyan/jars/sedona-spark-shaded-3.0_2.12-1.6.1.jar'
geotools_jar = '/home/jovyan/jars/geotools-wrapper-1.7.0-28.5.jar'
spark = (
    SparkSession.builder
        .config("spark.jars", f"{sedona_jar},{geotools_jar}") 
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
        .config("spark.kryo.registrator", SedonaKryoRegistrator.getName)
        .config("spark.sql.extensions", "org.apache.sedona.sql.SedonaSqlExtensions")
        .appName('NYC Taxi')    
        .getOrCreate()
)

In [3]:
# Load trip data

df_trip = spark.read.csv('../data/Sample NYC Data.csv', header=True, inferSchema=True)

In [4]:
# Load spatial data of NY

# First we use geopandas to read the geojson
gdf = gpd.read_file('../data/nyc-boroughs.geojson')

# Convert geom to WKT
gdf['geom'] = gdf['geometry'].apply(lambda geom: geom.wkt if geom else None)

# geopandas df to pandas df
pdf = gdf.astype(str)

# pandas df to spark df
df_geom = spark.createDataFrame(pdf)

# Convert WKT to geom (format) which is used for intersections
df_geom = df_geom.withColumn("geom", expr("ST_GeomFromWKT(geom)"))

In [5]:
# Helper function to convert long and lat to Point

udf_to_point = udf(lambda lon, lat: Point(lon, lat).wkt if lon is not None and lat is not None else '')

In [6]:
# Trip df to WKT -> geom 

df_trip_w_points = (df_trip
     .withColumn('pickup_point', udf_to_point(col('pickup_longitude'), col('pickup_latitude')))
     .withColumn('dropoff_point', udf_to_point(col('dropoff_longitude'), col('dropoff_latitude')))
     .withColumn('geom_pickup', expr('ST_GeomFromWKT(pickup_point)'))
     .withColumn('geom_dropoff', expr('ST_GeomFromWKT(dropoff_point)'))
    )

In [7]:
# Join geospatial data to trip pickup locations

df_trip_w_prickup = df_geom.alias('geo').join(
    df_trip_w_points.alias('travel'),
    expr('ST_Intersects(geo.geom, travel.geom_pickup)'),
    'inner'
)

In [8]:
# Distinguish pickup borough
df_trip_w_prickup = df_trip_w_prickup.withColumn('pickup_borough', col('borough'))

# Remove excess columns
df_trip_w_prickup = df_trip_w_prickup.drop("geometry", "@id", "geom", "borough", "boroughCode")

In [9]:
# Join geospatial data to trip dropoff locations

df_trip_w_prickup_n_dropoff = df_geom.alias('geo').join(
    df_trip_w_prickup.alias('travel'),
    expr('ST_Intersects(geo.geom, travel.geom_dropoff)'),
    'inner'
)

In [10]:
# Distinguish dropoff borough
df_trip_w_prickup_n_dropoff = df_trip_w_prickup_n_dropoff.withColumn('dropoff_borough', col('borough'))

# Remove excess columns
df_trip_w_prickup_n_dropoff = df_trip_w_prickup_n_dropoff.drop("geometry", "@id", "geom", "borough", "boroughCode")

In [11]:
# Test that the solution works

df_trip_w_prickup_n_dropoff.first()

Row(medallion='F32C996F49594F21C3C14A9E1821A81A', hack_license='04BF366C306B7476F390E96B5E3717E2', vendor_id='VTS', rate_code=1, store_and_fwd_flag=None, pickup_datetime='13-01-13 03:10', dropoff_datetime='13-01-13 03:37', passenger_count=2, pickup_longitude=-74.074799, pickup_latitude=40.645111, dropoff_longitude=-74.177147, dropoff_latitude=40.540531, pickup_point='POINT (-74.074799 40.645111)', dropoff_point='POINT (-74.177147 40.540531)', geom_pickup=<POINT (-74.075 40.645)>, geom_dropoff=<POINT (-74.177 40.541)>, pickup_borough='Staten Island', dropoff_borough='Staten Island')

In [12]:
# Remove columns that were used for intersection

df_save = df_trip_w_prickup_n_dropoff.drop("pickup_point", "dropoff_point", "geom_pickup", "geom_dropoff")

In [13]:
# Save the result to parquet to use later for queries

df_save.write.parquet("../output/output.parquet")